<a href="https://colab.research.google.com/github/ngzhiwei517/Water-Volume-Prediction-for-Lakes-and-Reservoirs/blob/main/water_volume_prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediciton of Lake Water Volume Changes

# 1.0 Introduction

# 1.1 Group information

Group Name: HydroDS (Group 15)

Group Member:
- 23054206 NG YU HENG
- 23005031 CHANG CAI TING
- 23005000 CHUA HUI MIN
- 23005227 KUEH PANG LANG
- 23051966 NG ZHI WEI
- 23004979 POH JING MIN

# 1.2 Problems and Solutions

Our project is about predicting the changes of water volume in 24 hours of a lake or a resevoir by the conditions of the weather.

Recently, Penang has been facing a critical issue where their water resevoirs are low on water. The water supply is only enough to last for another 30 days. These show that the people working at the water resevoirs fail to have an effective long term plan when dealing with climate changes to make sure that the water level in the dam is sufficient for supplying to the people. 10 years ago, Cameron Highlands experienced a tragedy where the water of the dam was released only when the water exceed its safety level due to heavy rain falls and caused a lighting flood. This show that the people working at the dam fail to take effective action when dealing with sudden changes of weather conditions.



From the situations above, we can see that weather is the major factor affecting the water volume in the water resevoirs or dam.

To solve the problems, we come up with two solutions:
- **Water volume changes prediction for unusual weather conditions**
- Water volume changes forecasting for climate changes

With the help of prediction and forecasting, it allow the responsible party to take early action to prevent the above issues to happen.

For this project, we will be focusing on **prediction for water volume changes in 24 hours**.

# 2.0 Data Mining

# 2.1 Lake dataset

After surfing through the Internet, we found that we were unable to find any historical data about the water level of the dam in Malaysia. The most we can get is the current data which is too little to train our model even if we start collecting it by April 2024. Hence, we decided to look for data out of Malaysia.

Luckily for us, we were able to find a website which contain the historical data regarding a few lakes located in Phoenix, AZ, USA. In order to extract those data out and store it in a .csv file, we have to perform Web Scrapping.

# 2.1.1 Inspecting the website (Lake dataset)

Before scrapping the data out from the website, we make some discovering about the website first and understand where the data is stored and how to extract the data for a period of time by performing the following code.

We need the help of some packages in order to perform web scrapping.
- BeautifulSoup
- requests

In [ ]:
# Import required packages
from bs4 import BeautifulSoup
import requests

First, we try to get request from the website.


---

Source: https://www.watershedconnection.com/

In [ ]:
# Website for data mining
url = "https://streamflow.watershedconnection.com/DWR?reportDate=2017-1-1"

In [ ]:
# Get requests from website
requests.get(url)

It returns us a response 200. Response 200 indicates that our request to the website has succeeded.

Now, we try to get the HTML of the website.

In [ ]:
# Get requests from website
page_lake_test = requests.get(url)

In [ ]:
# Get the html of the website
soup_lake_test = BeautifulSoup(page_lake_test.text, 'html')

In [ ]:
# Print the html of the website
print(soup_lake_test.prettify())

Well, there is quite a lot of code in the HTML. By inspecting the website, we know that the table containing the data is code under the tags 'table'. Let's have a search for the tags 'table'.

In [ ]:
# Find all 'table' in the html (Searching for the tags containing the table of the data)
soup_lake_test.find_all('td')

There is the tags 'table' containing the data we want. However, there are also other tables in that website too. We select the table we want by adding a '[0]' at the end of the previous code.

In [ ]:
# Find all 'table' in the html (Searching for the tags containing the table of the data)
soup_lake_test.find_all('table')[0]

The table we wanted has been selected. From the code, we able to see that the data we wanted is under the tags 'td'. Let's try to get the data out.

In [ ]:
# Find all 'td' in the html (Searching for the data we wanted)
soup_lake_test.find_all('td')

Let's pull the first data out from the tags to see whether the data is in the format that we wanted.

In [ ]:
# Pulling text information from the website
soup_lake_test.find('td').text

Well, almost there but it containing something that we don't want. Let's trim it off.

In [ ]:
# Pulling text information from the website
soup_lake_test.find('td').text.strip()

Nice, this is what we wanted.

The websites also containing some information of the weather for that respective day. From the inspection of the websites, the weather data is code under the 'b' tags.

In [ ]:
# Find all 'b' in the html (Searching for the weather data)
soup_lake_test.find_all('b')

Each data is in the format highest/lowest. So, let us try to separate the first data.

In [ ]:
# Separating the data and removing unwanted value from the data
temperature_test = soup_lake_test.find_all('b')[0]
temperature_list_test = temperature_test.text.strip().split("°")
temperature_list_test[1] = temperature_list_test[1].replace("/", "")
temperature_list_test

From the above list, we know that the data one is the first and the second items in the list.

Since we know where our data is located in the website, it's time to extract those data and store it in a dataframe.

In [ ]:
# Selecting the table needed
table_lake_test = soup_lake_test.find_all('table')[0]

Before creating the dataframe, we need to set the feature names first. The feature names are under the tags 'th'. Let's have a look of it.

In [ ]:
# Find all 'th' in the html (Searching for the feature names)
soup_lake_test.find_all('th')

That was an unexpected coding. We have no choice to set the feature names ourself.

In [ ]:
# Setting the feature names manually into a list
feature_names_lake_test = ['Location', 'Full (%)', 'Current Elevation(ft)',
                      'Current Storage (af)', 'Remaining Elevation (ft)',
                      'Available Storage (af)', '24 hr. Change',
                      'Rain (inches)']
feature_names_lake_test

This is better. Now we will create a dataframe and set the feature names into it.

In [ ]:
import pandas as pd

In [ ]:
lake_test_df = pd.DataFrame(columns = feature_names_lake_test)
lake_test_df

Looks good up until here.

Now it is time to abtract the data we wanted and storing it into the dataframe.

In [ ]:
column_data_lake_test = table_lake_test.find_all('tr')

In [ ]:
for row in column_data_lake_test[2:]:
  row_data = row.find_all('td')
  individual_row_data = [data.text.strip() for data in row_data]

  length = len(lake_test_df)
  lake_test_df.loc[length] = individual_row_data

lake_test_df

Seems like the web scrapping works for this website as the data we wanted is successfully extract from the website into the dataframe. Now, we will be moving to the next step which is extracting data for a range of time.

# 2.1.2 Extracting data for a range of time (Lake dataset)

Setting up the date range for the data we wanted to extract.

In [ ]:
# Import package needed for convertion between text and date
from datetime import datetime, timedelta

In [ ]:
# Set Date range
date_start_text = "2013-1-1"
date_end_text = "2023-1-1"
freq_text = "1"

The range of our data gathering will be daily for a period of 10 years.

The convertion from text to date is necessary to be put as the range of the while loop.

In [ ]:
# Convert text to date
date_start_inc = date_start_text
date_start_inc_date = datetime.strptime(date_start_inc, "%Y-%m-%d")
date_end_date = datetime.strptime(date_end_text, "%Y-%m-%d")
freq_obj = int(freq_text)

date_start_inc_date = date_start_inc_date.date()
date_end_date = date_end_date.date()

Setting up the feature names needed for the dataframe.

In [ ]:
# Setting up features name
feature_names_lake = [ 'Location', 'Full_%', 'Current_Elevation_ft',
                      'Current_Storage_af', 'Remaining_Elevation_ft',
                      'Available_Storage_af', '24hr.Change',
                      'Rain_inches', 'Date', 'Highest_temperature',
                       'Lowest_temperature','Highest_humidity',
                       'Lowest_humidity']
feature_names_lake

There will be a total of 13 features for this dataset

In [ ]:
# Setting up the dataframe to store data later with the feature names we set previously
lake_df = pd.DataFrame(columns = feature_names_lake)

Our dataframe is ready to store data gather from the website

In [ ]:
# Using the while loop to gather data for a period of time
while date_start_inc_date <= date_end_date:
  # Date will be automatically insert to the back of the url to access data for that particular day
  url = 'https://streamflow.watershedconnection.com/DWR?reportDate='+ date_start_inc_date.strftime("%Y-%m-%d")
  page_lake = requests.get(url)
  soup_lake = BeautifulSoup(page_lake.text, 'html')
  table_lake = soup_lake.find_all('table')[0]
  column_data_lake = table_lake.find_all('tr')
  for row_lake in column_data_lake[2:]:
    row_data_lake = row_lake.find_all('td')
    individual_row_data_lake = [data.text.strip() for data in row_data_lake]
    individual_row_data_lake.append(date_start_inc_date.strftime("%Y-%m-%d"))
    # Triming the temperature value and adding it into the list
    temperature = soup_lake.find_all('b')[0]
    temperature_list = temperature.text.strip().split("°")
    temperature_list[1] = temperature_list[1].replace("/", "")
    individual_row_data_lake.append(temperature_list[0])
    individual_row_data_lake.append(temperature_list[1])
    # Triming the humidity value and adding it into the list
    humidity = soup_lake.find_all('b')[2]
    humidity_list = humidity.text.strip().split()
    individual_row_data_lake.append(humidity_list[0])
    individual_row_data_lake.append(humidity_list[2])
    # Storing the list of data into the dataframe
    length = len(lake_df)
    lake_df.loc[length] = individual_row_data_lake

  date_start_inc_date = date_start_inc_date + timedelta(days = freq_obj)

Having a look at the dataframe.

In [ ]:
# Print out a dataframe
lake_df

From the table above, the data we gather has correctly store in their respective column. A total of 36530 rows of data gather for 10 years period

In [ ]:
lake_df.to_csv(r'/content/sample_data/lake_dataset_10years.csv', index = False)

We save the data gather into a csv file.

After inspecting the dataset, we found out the weather information gather from that website is not precise enough. Hence, we decided to gather the information of the weather from the nearest weather station to the area where the lakes are located from other website.

# 2.2 Weather dataset

First we need to see which weather station is the nearest to that area. Only the weather station at the airport in that area is storing the historical weather data.

The airport available there is as follow:
- Phoenix Sky Harbor International Airport
- Phoenix-Mesa Gateway Airport
- Falcon Field Airport
- Scottsdale Airport
- Phoenix Deer Valley Airport
- Glendale Municipal Airport
- Phoenix-Goodyear Airport


We have chosen Falcon Field Airport since it is nearest to that area.

We now gather the data about the weather information of that area from this website.

---


Source: https://www.visualcrossing.com/

# 2.2.1 Inspecting the website (Weather dataset)

After taking a tour through this website, it's provide API that allow user to get the data from their website easily. However, for free user, a total of 1000 records are allowed for a day. This mean that we need a few day to gather all the data we wanted.

Since API service is provided, it save up a lot of our works to inspect the website.

# 2.2.2 Extracting data for a range of time (Weather dataset)

As same as previous, we first need to indicate the range of date of the data we wanted to extract. Since we are only able to extract limited data per day, in this notebook, we will only be extracting data for a period of one year.

In [ ]:
# Set Date range
date_start_text = "2013-1-1"
date_end_text = "2014-1-1"
freq_text = "1"

Converting the date from string into date

In [ ]:
# Convert text to date
date_start_inc = date_start_text
date_start_inc_date = datetime.strptime(date_start_inc, "%Y-%m-%d")
date_end_date = datetime.strptime(date_end_text, "%Y-%m-%d")
freq_obj = int(freq_text)

date_start_inc_date = date_start_inc_date.date()
date_end_date = date_end_date.date()

We now start to extract the data we wanted using the API. We will be getting weather information for a day for each query we make. Hence, we need to create a list to store all the dataframe we get so that we can combine it into one later.

In [ ]:
# Creating the list to store the .csv file
list_of_csv = []
# Creating the while loop
while date_start_inc_date <= date_end_date:
  url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/%20Falcon%20Field%20Airport%2C%204800%20E%20Falcon%20Dr%2C%20Mesa/'+date_start_inc_date.strftime("%Y-%m-%d")+'/' +date_start_inc_date.strftime("%Y-%m-%d")+'?unitGroup=metric&include=days&key={API KEY}&contentType=csv'
  df = pd.read_csv(url, index_col=None, header = 0)
  date_start_inc_date = date_start_inc_date + timedelta(days = freq_obj)
  # Adding the dataframe into the list
  list_of_csv.append(df)

We proceed with the combination of the dataframe

In [ ]:
# Combining all the dataframe in the list into one
weather_2013_df = pd.concat(list_of_csv, axis= 0,ignore_index=True)

Let's have a glance through the dataframe we combined

In [ ]:
# Print the dataset
weather_2013_df

Everything looks good. Time to save it into a .csv file for later use.

In [ ]:
# Saving the dataframe into a .csv file
weather_2013_df.to_csv(r'/content/sample_data/weather_dataset_2013.csv', index = False)

Now, we have the weather dataset for the year 2013. The dataset for the remaining years will be done using another notebook by using the same code as above.

Since all of the dataset we wanted are ready, it's time to proceed to the next section,which is Data Preprocessing.

# 3.0 Data cleaning and preprocessing

Data cleaning is the initial phase of refining dataset, making it readable and usable with techniques like removing and usable with techniques such as handling missing values, changing data types, removing duplicates etc.

Data preprocessing take place to refine data and scaling with more advanced techniques such as encoding categorical variables. handling outliers in order to achieve a better results.

# 3.1 Data loading

 Import lake dataset

In [ ]:
import pandas as pd

# Importing lake dataset
lake_10years_df=pd.read_csv("/content/lake_dataset_10years.csv")


Import weather dataset from year 2013 to 2022

In [ ]:
# Importing dataset

# Each dataframe have been predefined to prevent conflict between the datafrane variable name of two different notebooks

# Importing weather dataset

weather_2013_df = pd.read_csv("/content/weather_dataset_cv_2013.csv")
weather_2014_df = pd.read_csv("/content/weather_dataset_cv_2014.csv")
weather_2015_df = pd.read_csv("/content/weather_dataset_cv_2015.csv")
weather_2016_df = pd.read_csv("/content/weather_dataset_cv_2016.csv")
weather_2017_df = pd.read_csv("/content/weather_dataset_cv_2017.csv")
weather_2018_df = pd.read_csv("/content/weather_dataset_cv_2018.csv")
weather_2019_df = pd.read_csv("/content/weather_dataset_cv_2019.csv")
weather_2020_df = pd.read_csv("/content/weather_dataset_cv_2020.csv")
weather_2021_df = pd.read_csv("/content/weather_dataset_cv_2021.csv")
weather_2022_df = pd.read_csv("/content/weather_dataset_cv_2022.csv")


To merge all the weather datasets into one dataframe, we created a list called list_of_csv to store DataFrame objects. The list contains DataFrame variables named weather_2013_df, weather_2014_df, and so on up to weather_2022_df, each presumably holding weather data for a specific year.

In [ ]:
# Creating the list to store the .csv file
list_of_csv = [weather_2013_df,weather_2014_df,weather_2015_df,weather_2016_df,
               weather_2017_df,weather_2018_df,weather_2019_df,weather_2020_df,
               weather_2021_df,weather_2022_df]

We combines all the DataFrames stored in the list_of_csv into a single DataFrame named weather_10years_df.

In [ ]:
# Combining all the dataframe in the list into one
weather_10years_df = pd.concat(list_of_csv, axis= 0,ignore_index=True)

Let have a look for our merged dataset

In [ ]:
# Print the dataset
weather_10years_df


#Optional
Dowload the combine csv file to your local repository

In [ ]:
# Save the combined DataFrame as a CSV file
weather_10years_df.to_csv('Weather_10years.csv', index=False)


In [ ]:
from google.colab import files

files.download('Weather_10years.csv')


# 3.2 Data cleaning

# 3.2.1 Overview the dataset

**Lake dataset**

In [ ]:
# Checking data
print("\nInformation of the dataset")
print(lake_10years_df.info())

In [ ]:
print("\nNumber of unique value for each column")
print(lake_10years_df.nunique())

In [ ]:
print("\nDetermining the data types")
print(lake_10years_df.dtypes)

All of the data extracted from the website are stored as String rather then integer. Convertion should be made as most of the data such as current elevation, current storage etc. should be integer.

In [ ]:
print(f'The dateset has {lake_10years_df.shape[0]} rows and {lake_10years_df.shape[1]} columns.')

From the information above, we know that the dataset contains 36530 rows of data and 13 features.

Identify the column

In [ ]:
print('Columns of dataset\n__________________________________________________________________________________________________')
# Print the columns of the dataset
print('Lake Dataset:', lake_10years_df.columns)

The above are the list of features name for the lake dataset.

**Weather dataset**

In [ ]:
#Checking data
print("\nPrint information of the dataset: ")
print(weather_10years_df.info())

In [ ]:
print("\nNumber of unique value for each column")
print(weather_10years_df.nunique())

In [ ]:
print("\nDetermining the data types")
print(weather_10years_df.dtypes)

There are some categorical data in the dataset that need to be encode later in the data preprocessing process.

In [ ]:
print(f'The dateset has {weather_10years_df.shape[0]} rows and {weather_10years_df.shape[1]} columns.')

From the information above, we know that the dataset contains 3662 rows of data and 33 features.

Identify the column

In [ ]:
print('Columns of dataset\n__________________________________________________________________________________________________')
# Print the columns of the dataset
print('Weather Dataset: ', weather_10years_df.columns)

The above are the list of features name of the weather dataset.

# 3.2.2 Lake data modifiying

We wanted to use the the previous day storage capacity as a feature to predict the change of volume of the lake in 24 hours. A small modifying to the dataset is needed. We will be creating a new column with the title 'Previous_Storage_af'.

We will use the data from the features, 'Current_Storage_af' and shift the data 10 rows downward. Now we will have a feature which contains the volume of the storage of the previous day.

In [ ]:
# Having a look at the lake dataset
lake_10years_df

In [ ]:
# Creating a new column named 'Previous_Storage_af'
# Using the data from column named 'Current_Storage_af', shifting it 10 rows downward, and store in that column
lake_10years_df.insert(3, "Previous_Storage_af", lake_10years_df["Current_Storage_af"].shift(10))

In [ ]:
# Printing the lake dataset again to see the added column
lake_10years_df.head(20)


A new feature is created by modifying one of the existing features. However, null values existed in this new features because of the shifting of data. It will be remove later in the data preprocessing process.

# 3.2.3 Rename the date column

We choose to rename the 'Date' column to 'datetime' in the lake DataFrame since we noticed that the weather dataset uses 'datetime' as the column name for the corresponding information. This adjustment ensures consistency across datasets and facilitates seamless data integration.

In [ ]:
# Renaming the 'Date' column to 'datetime'
lake_10years_df.rename(columns={'Date': 'datetime'}, inplace=True)

# Check the updated column names
print("\nFeatures of the dataset ")
print(lake_10years_df.columns)


# 3.2.4 Choosing a lake location and removing the others

From the lake datasets, it contains a few lake locations that can be selected for prediction. However, including all the locations will affect the accuracy of the prediction.

Hence, a discussion has been made and we decided to select one location only which is Roosevelt Lake (Roosevelt Dam) for our prediction.

In [ ]:
# Removing unwanted locations
lake_10years_df = lake_10years_df.drop(lake_10years_df[lake_10years_df['Location'] == 'Apache Lake (Horse Mesa Dam)'].index)
lake_10years_df = lake_10years_df.drop(lake_10years_df[lake_10years_df['Location'] == 'Canyon Lake (Mormon Flat Dam)'].index)
lake_10years_df = lake_10years_df.drop(lake_10years_df[lake_10years_df['Location'] == 'Saguaro Lake (Stewart Mountain Dam)'].index)
lake_10years_df = lake_10years_df.drop(lake_10years_df[lake_10years_df['Location'] == 'Horseshoe Lake (Horseshoe Dam)'].index)
lake_10years_df = lake_10years_df.drop(lake_10years_df[lake_10years_df['Location'] == 'Bartlett Lake (Bartlett Dam)'].index)

In [ ]:
# Having a look at the lake dataset
lake_10years_df

# 3.2.5 Replace Missing Values for selected features

To avoid losing too much potential data when removing null value by using the command dropna(), we decided to replace the null value with suitable value for some features.

In [ ]:
print("\nFind missing value of each column")
print(lake_10years_df.isna().sum())

print("\nFind missing value of each column")
print(weather_10years_df.isna().sum())

In [ ]:
# Replace missing values in the 'precitype' column with 'none'
weather_10years_df['preciptype'].fillna('none', inplace=True)

# Replace missing values in the 'snow' column with mean
weather_10years_df['snow'].fillna(weather_10years_df['snow'].mean(), inplace=True)

# Replace missing values in the 'snowdepth' column with mean
weather_10years_df['snowdepth'].fillna(weather_10years_df['snowdepth'].mean(), inplace=True)

# Replace missing values in the 'windgust' column with 0
weather_10years_df['windgust'].fillna(0, inplace=True)

# Replace missing values in the 'solarradiation' column with mean
weather_10years_df['solarradiation'].fillna(weather_10years_df['solarradiation'].mean(), inplace=True)

# Replace missing values in the 'solarenergy' column with mean
weather_10years_df['solarenergy'].fillna(weather_10years_df['solarenergy'].mean(), inplace=True)

# Replace missing values in the 'uvindex' column with mean
weather_10years_df['uvindex'].fillna(weather_10years_df['uvindex'].mean(), inplace=True)

# Replace missing values in the 'severerisk' column with 0
weather_10years_df['severerisk'].fillna(0, inplace=True)



# Display the first few rows to verify the changes
print(weather_10years_df.head())

In [ ]:
print("\nFind missing value of each column")
print(lake_10years_df.isna().sum())

print("\nFind missing value of each column")
print(weather_10years_df.isna().sum())

# 3.2.6 Handle Missing Value Which Are Not Replaced with Any Value

Find the missing value of each column to ensures that subsequent analyses or modeling efforts are based on reliable and complete data.



In [ ]:
print("\nFind missing value of each column")
print(lake_10years_df.isna().sum())

print("\nFind missing value of each column")
print(weather_10years_df.isna().sum())

We can obseve that there are null values present in some columns of the dataset, such as "Current_Elevation_ft", "Remaining_Elevation_ft", "Rain_inches", and others. Handling null values is essential to ensure the integrity of the data and prevent issues during analysis.

Remove all the row with missing data

In [ ]:
print("\nRemove all rows with missing data by using dropna()")
lake_10years_df = lake_10years_df.dropna ()
print(lake_10years_df.isna().sum())

print("\nRemove all rows with missing data by using dropna()")
weather_10years_df = weather_10years_df.dropna ()
print(weather_10years_df.isna().sum())


We had sucessfully remove all the missing data

In [ ]:
lake_10years_df.info()

In [ ]:
print(f'The dateset has {lake_10years_df.shape[0]} rows and {lake_10years_df.shape[1]} columns.')

For lake dataset, we have 3652 rows of data left after removing the null values.

In [ ]:
weather_10years_df.info()

In [ ]:
print(f'The dateset has {weather_10years_df.shape[0]} rows and {weather_10years_df.shape[1]} columns.')

For weather dataset, we have 3662 rows of data left after removing the null values.

# 3.2.7 Identify and changing incorrect data types

**Lake dataset**

In [ ]:
# Get data types of each column
data_types = lake_10years_df.dtypes

# Separate columns into categorical and numerical variables
categorical_vars = data_types[data_types == 'object'].index.tolist()
numerical_vars = data_types[data_types != 'object'].index.tolist()

# Print categorical and numerical variables
print("Categorical variables:")
print(categorical_vars)
print("\nNumerical variables:")
print(numerical_vars)


**Weather dataset**

In [ ]:
# Get data types of each column
data_types = weather_10years_df.dtypes

# Separate columns into categorical and numerical variables
categorical_vars = data_types[data_types == 'object'].index.tolist()
numerical_vars = data_types[data_types != 'object'].index.tolist()

# Print categorical and numerical variables
print("Categorical variables:")
print(categorical_vars)
print("\nNumerical variables:")
print(numerical_vars)

We want to convert the columns from "object" data type to numerical data types (integers or floats) because all of the data except 'Location' and 'datetime' represent numerical values. Although the columns are currently labeled as "object," the data within them should be numeric.

**Lake dataset**

In [ ]:
import pandas as pd

# Convert columns to numerical data types
numerical_columns = ['Full_%','Previous_Storage_af', 'Current_Storage_af', 'Available_Storage_af', '24hr.Change', 'Rain_inches', 'Highest_temperature', 'Lowest_temperature', 'Highest_humidity', 'Lowest_humidity']
numerical_columns1 = ['Current_Elevation_ft', 'Remaining_Elevation_ft']

# Convert columns with float data types
for col in numerical_columns1:
    lake_10years_df[col] = pd.to_numeric(lake_10years_df[col].astype(str).str.replace(',', ''), errors='coerce')

# Convert columns with integer data types
for col in numerical_columns:
    lake_10years_df[col] = pd.to_numeric(lake_10years_df[col].astype(str).str.replace(',', ''), errors='coerce', downcast='integer')

After converting, identify categorical variables and numerical variables again

In [ ]:
# Get data types of each column
data_types = lake_10years_df.dtypes

# Separate columns into categorical and numerical variables
categorical_vars = data_types[data_types == 'object'].index.tolist()
numerical_vars = data_types[data_types != 'object'].index.tolist()

# Print categorical and numerical variables
print("Categorical variables:")
print(categorical_vars)
print("\nNumerical variables:")
print(numerical_vars)


In [ ]:
lake_10years_df.info()

Since we observe that there are numerical value again after changing the type, we decide to remove the null values again.

In [ ]:
print("\nRemove all rows with missing data by using dropna()")
lake_10years_df = lake_10years_df.dropna ()
print(lake_10years_df.isna().sum())
lake_10years_df.info()

In [ ]:
print(f'The dateset has {lake_10years_df.shape[0]} rows and {lake_10years_df.shape[1]} columns.')

After removing the null values, we have 3438 rows of data in our lake dataset.

# 3.2.8 Identify and handle duplicate value

**Lake dataset**

In [ ]:
# Check for duplicate rows
duplicate_rows = lake_10years_df[lake_10years_df.duplicated()]

# Check if there are any duplicate rows
if duplicate_rows.empty:
    print("There are no duplicate rows.")

else:
   print("Duplicate values are found!")

**Weather dataset**

In [ ]:
# Check for duplicate values in the 'datetime'
duplicate_datetime = weather_10years_df['datetime'].duplicated().any()

# Check if there are any duplicate values in the 'datetime'
if duplicate_datetime:
    print("Duplicate values found in the 'datetime' column.")

    duplicate_count = weather_10years_df['datetime'].duplicated().sum()
    # Display the number of duplicate value
    print("Number of duplicate values in 'datetime' column:", duplicate_count)
else:
    print("No duplicate values found in the 'datetime' column.")



We remove the duplicate values in 'datetime' column to ensure that our models are trained on clean and accurate data, which can lead to better predictive performance and generalization on unseen data.

In [ ]:
# Remove duplicate rows based on 'datetime' column
weather_10years_df.drop_duplicates(subset=['datetime'], keep='first', inplace=True)

print(f'The dateset has {weather_10years_df.shape[0]} rows and {weather_10years_df.shape[1]} columns.')

After removing duplicate value, we left with 3653 rows of data in weather dataset.

In [ ]:
weather_10years_df.info()

# 3.3 Data Integration



To merge the weather and lake datasets into one, we'll first import both datasets. Then, we'll merge them based on a common identifier, such as date or location. Here's a step-by-step guide:

**Lake dataset details:**

In [ ]:
# Check the details of the lake dataset
print(lake_10years_df.info())

print("\nFirst few rows of lake dataset")
print (lake_10years_df.head())

**Weather dataset details:**


In [ ]:
# Check the details of the weather dataset
print(weather_10years_df.info())

print("\nFirst few rows of weather dataset")
print (weather_10years_df.head())

**Dataset after merging**



In [ ]:
# Merging of DataFrame using the pd.merge ()
merge_dataset_df = pd.merge(lake_10years_df, weather_10years_df, on = "datetime")
merge_dataset_df

#Optional
Dowload the combine csv file to your local repository

In [ ]:
# Save the combined DataFrame as a CSV file
merge_dataset_df.to_csv('merge_dataset.csv', index=False)


In [ ]:
from google.colab import files

files.download('merge_dataset.csv')


# 3.4 Exploratory data analysis

Exploratory data analysis is used to analyze and investigate dataset and summarize their main characteristics.

Exploratory data analysis allows us to have a better understanding of the pattern of the dataset, spot anomalies, test hypothesis and check assumptions. It will ease our works when dealing with data preprocessing and model selection.

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Use this to do data analysis
test_df = merge_dataset_df

In [ ]:
# Display the first few rows of the merged dataset
print("First few rows of the merged dataset:")
test_df

In [ ]:
# Display information about the dataset
print("\nMereged dataset Information:")
print(test_df.info())

In [ ]:
# Summary statistics of numerical features
print("\nSummary Statistics of Numerical Features:")
test_df.describe()

In [ ]:
# List column names
print("\nColumn names:")
print(test_df.columns)

In [ ]:
# Inspect data types
print("\nData types:")
print(test_df.dtypes)

In [ ]:
print("\nFind missing value of each column")
print(test_df.isna().sum())

The dataset contains no null value.

In [ ]:
test_df.nunique()

The features 'Location', 'name, and 'snow' only has 1 unique value, hence it is redundant to have them in the dataset. These features will be removed later in the data preprocessing process.

In [ ]:
test_df.sort_values(by = '24hr.Change', ascending = False).head(10)

In [ ]:
# Get data types of each column
data_types = test_df.dtypes

# Separate columns into categorical and numerical variables
categorical_vars = data_types[data_types == 'object'].index.tolist()
numerical_vars = data_types[data_types != 'object'].index.tolist()

# Print categorical and numerical variables
print("Categorical variables:")
print(categorical_vars)
print("\nNumerical variables:")
print(numerical_vars)

The dataset contains some categorical variables that need to be encoded to numerical variable later in the data preprocessing process.

In [ ]:
import seaborn
import matplotlib.pyplot as plt

seaborn.pairplot(test_df)
plt.show()

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(test_df['datetime'].astype('datetime64[ns]'), test_df['24hr.Change'], label = '24hr change')
plt.xlabel('Date')
plt.ylabel('24hr changes')
plt.legend()
plt.show()

From the line graph above, we can see that there are some outliers existed in the 24hr changes data. These outliers indicates that there exists unexpected changes of water volume happened in some of the days.

After removing the outliers, it will helps us to get a more accurate prediction for the 24hr changes. A more consistent 24hr changes is also pretty useful for forecasting purpose.



In [ ]:
import seaborn as sns

# Select only numeric columns for correlation matrix
numeric_cols = merge_dataset_df.select_dtypes(include=['float64', 'int64']).columns
merge_dataset_numeric_df = merge_dataset_df[numeric_cols]

# Calculate the correlation matrix
correlation_matrix = merge_dataset_numeric_df.corr()

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap of Correlation Matrix')
plt.show()



Colors indicate the degree and direction of correlation: warmer colors (e.g., red) denote positive correlations, while cooler colors (e.g., blue) indicate negative correlations.

- The temperature related features have strong correlation with each other.
- The temperature related features are slightly correlated to solar radiation and solar energy.
- The temperature related features have negative correlation with humidity.
- Features in weather dataset are positively correlated to each other except for remaining elevation and available storage.

# 3.5 Data preprocessing

# 3.5.1 Handling Outliers

Visualise boxplot to check for outliers

**Lake dataset**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette("Set3")
fig, axs = plt.subplots(4, 3, figsize=(12, 10))
numerical_features = ['Full_%', 'Current_Elevation_ft', 'Previous_Storage_af','Current_Storage_af',
                      'Remaining_Elevation_ft', 'Available_Storage_af', '24hr.Change',
                      'Rain_inches', 'Highest_temperature', 'Lowest_temperature',
                      'Highest_humidity', 'Lowest_humidity']  # Added the two new features

for i, feature in enumerate(numerical_features):
    sns.boxplot(data=lake_10years_df, x=feature, orient='h', ax=axs[i // 3, i % 3])
    axs[i // 3, i % 3].set_title(feature)

plt.suptitle("Before Outliers are Removed", fontsize=16)
plt.tight_layout()
plt.show()


After analyzing the features 'previous_Storage_af','Current_Storage_af','Available_Storage_af', '24hr.Change', 'Rain_inches' and 'Lowest_humidity', it becomes evident that these features contain outlier values that significantly deviate from the majority of the data points. Outliers are data points that lie far away from the rest of the dataset, and they can distort statistical analyses and machine learning models, leading to inaccurate results.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the Seaborn palette
sns.set_palette("Set3")

# Features for which outliers will be removed
features_to_remove_outliers = ['Previous_Storage_af','Current_Storage_af', 'Available_Storage_af', '24hr.Change', 'Rain_inches', 'Lowest_humidity']

# Remove outliers for specified features
for feature in features_to_remove_outliers:
    Q1 = merge_dataset_df[feature].quantile(0.25)
    Q3 = merge_dataset_df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Calculating the lower and upper fences
    Lower_Fence = Q1 - (1.5 * IQR)
    Upper_Fence = Q3 + (1.5 * IQR)

    # Removing outliers
    merge_dataset_df = merge_dataset_df[(merge_dataset_df[feature] >= Lower_Fence) & (merge_dataset_df[feature] <= Upper_Fence)]

# Now, lake_data contains the dataset after removing outliers for the specified features

# Define the numerical features (including the cleaned ones)
numerical_features = ['Previous_Storage_af','Current_Storage_af', 'Available_Storage_af', '24hr.Change', 'Rain_inches', 'Lowest_humidity']

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Plot boxplots for each feature using the cleaned dataset
for i, feature in enumerate(numerical_features):
    sns.boxplot(data=merge_dataset_df, x=feature, orient='h', ax=axs[i // 3, i % 3])
    axs[i // 3, i % 3].set_title(feature)

# Add title
plt.suptitle("After Outliers are Removed", fontsize=16)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()


**Weather dataset**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette("Set3")
fig, axs = plt.subplots(4, 6, figsize=(18, 12))
numerical_features1 = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'moonphase']


for i, feature in enumerate(numerical_features1):
    sns.boxplot(data=weather_10years_df, x=feature, orient='h', ax=axs[i // 6, i % 6])
    axs[i // 6, i % 6].set_title(feature)

plt.suptitle("Before Outliers are Removed", fontsize=16)
plt.tight_layout()
plt.show()

After analyzing the features 'feelslikemin', 'humidity','precip', 'precipprob', 'windspeed', 'sealevelpressure', 'cloudcover', 'uvindex', 'severerisk', 'precipcover' and 'windgust', it becomes evident that these features contain outlier values that significantly deviate from the majority of the data points. Outliers are data points that lie far away from the rest of the dataset, and they can distort statistical analyses and machine learning models, leading to inaccurate results.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the Seaborn palette
sns.set_palette("Set3")

# Features for which outliers will be removed
features_to_remove_outliers = ['feelslikemin', 'humidity','precip', 'precipprob', 'precipcover','windgust', 'windspeed', 'sealevelpressure', 'cloudcover', 'uvindex', 'severerisk']

# Remove outliers for specified features
for feature in features_to_remove_outliers:
    Q1 = merge_dataset_df[feature].quantile(0.25)
    Q3 = merge_dataset_df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Calculating the lower and upper fences
    Lower_Fence = Q1 - (1.5 * IQR)
    Upper_Fence = Q3 + (1.5 * IQR)

    # Removing outliers
    merge_dataset_df = merge_dataset_df[(merge_dataset_df[feature] >= Lower_Fence) & (merge_dataset_df[feature] <= Upper_Fence)]

# Now, weather_data contains the dataset after removing outliers for the specified features

# Define the numerical features (including the cleaned ones)
numerical_features1 = ['feelslikemin', 'humidity','precip', 'precipprob', 'precipcover','windgust', 'windspeed', 'sealevelpressure', 'cloudcover', 'uvindex', 'severerisk']

# Create subplots
fig, axs = plt.subplots(3, 4, figsize=(18, 12))

# Plot boxplots for each feature using the cleaned dataset
for i, feature in enumerate(numerical_features1):
    sns.boxplot(data=merge_dataset_df, x=feature, orient='h', ax=axs[i // 4, i % 4])
    axs[i // 4, i % 4].set_title(feature)

# Add title
plt.suptitle("After Outliers are Removed", fontsize=16)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
merge_dataset_df

After removing the outlieres from our datasets, we left we 2171 rows of data.

# 3.5.2 Label Encoding

Before training the model with our dataset, we need to encode the categorical data into numerical data. To do so, we will be using Label Encoding.

In [ ]:
# Read Dataset and import LabelEncoder from sklearn.preprocessing package
from sklearn.preprocessing import LabelEncoder
import pandas as pd

print (merge_dataset_df.head())

# Select Non-Numerical Columns
data_columns_category = ['Location', 'datetime', 'name', 'preciptype', 'sunrise', 'sunset', 'conditions', 'description', 'icon', 'stations']
print (data_columns_category)
print (merge_dataset_df[data_columns_category].head())



In [ ]:
# Iterate through column to convert to numeric data using LabelEncoder ()
label_encoder = LabelEncoder()
for col in data_columns_category:
    merge_dataset_df[col] = label_encoder.fit_transform (merge_dataset_df[col])

print("Label Encoder Data:")
print(merge_dataset_df.head())

All categorical data has been encoded to numerical data.

# 3.5.3 Transforming Data of Different Scale

 Here we used the Min-Max scaling that transforms the features to a fixed range, between 0 and 1. It does this by subtracting the minimum value of the feature and then dividing by the range (the maximum value minus the minimum value).

In [ ]:
from sklearn import preprocessing
import pandas as pd

# Create the MinMaxScaler object
min_max_scaler = preprocessing.MinMaxScaler()

# Fit and transform the numerical columns using Min-Max scaling
merge_dataset_scaled_df = min_max_scaler.fit_transform(merge_dataset_df)
merge_dataset_scaled_df = pd.DataFrame(merge_dataset_scaled_df, columns = merge_dataset_df.columns)

# Display the scaled data
merge_dataset_scaled_df


# 4.0 Feature Selection / Extraction

For the weather dataset, we only needed the previous storage and rain inches as the features and 24 hours changes will be our target. The other features will be dropped from our dataset.

We also remove the 'snow' and 'snowdepth' column because Arizona is lack of snow due to its warm climate and low humidity. Most of Arizona has a desert climate with hot summers and mild winters.

The feature 'name' will be removed too as it is redundant.

This adjustment reduces the dimensionality and improve the model's generalization ability.

In [ ]:
droping = ['Location', 'Full_%', 'Current_Elevation_ft', 'Current_Storage_af', 'Remaining_Elevation_ft', 'Available_Storage_af',
      'Highest_temperature', 'Lowest_temperature', 'Highest_humidity', 'Lowest_humidity', 'name', 'snow', 'snowdepth']

for col in droping:
  merge_dataset_scaled_df = merge_dataset_scaled_df.drop(col, axis = 1)

In [ ]:
X = merge_dataset_scaled_df.drop('24hr.Change', axis = 1)
y = merge_dataset_scaled_df['24hr.Change']

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and testing sets
X_train_ns, X_test_ns, y_train_ns, y_test_ns = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# Print the shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Testing set shape:", X_test.shape)

# 5.0 Model Selection

# 5.1 Multiple linear regression model

Multiple linear regression is a statistical technique used to model the relationship between a dependent variable and multiple independent variables.Since we want to predict the 24-hour change in water level based on various factors.We decide to use multiple linear regression model.Multiple linear regression model allows us to assess how each independent variable contributes to changes in the dependent variable.

**Train the model by using traning dataset**

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Train the model
print("Training the Multiple Regression Model for Predicting 24hr.Change of Water Level")
model = LinearRegression()
model.fit(X_train, y_train)

# Plot the actual vs. predicted values on the training set
fig, ax = plt.subplots()
ax.scatter(y_train, model.predict(X_train), alpha=0.5, label='Data Points')
ax.set_xlabel('Actual 24hr.Change of Water Level (Training Set)')
ax.set_ylabel('Predicted 24hr.Change of Water Level (Training Set)')
ax.set_title('Actual vs. Predicted Values (Training Set)')

# Add the straight line representing the ideal case
min_val = min(y_train.min(), model.predict(X_train).min())
max_val = max(y_train.max(), model.predict(X_train).max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideal Line')

ax.legend()
plt.show()

In [ ]:
# Get the predicted values on the training set
y_train_pred = model.predict(X_train)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
print(f"Mean Squared Error (Training Set): {mse_train}")

# Calculate Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
print(f"Root Mean Squared Error (Training Set): {rmse_train}")

# Calculate R-squared
r2_train = r2_score(y_train, y_train_pred)
print(f"R-squared (Training Set): {r2_train}")

# Histogram of residuals (training set)
sns.displot((y_train - model.predict(X_train)), bins=50)
plt.xlabel('Residuals (Training Set)')
plt.ylabel('Density')
plt.title('Histogram of Residuals (Multiple Regression)')
plt.show()

Since the histogram of residuals on the training set is peaked around 0.02 instead of 0.0, it could indicate a potential issue or opportunity for improvement in the linear regression model.The peak at 0.02 may indicate that the model has a slight bias, consistently over-predicting or under-predicting the target variable by a small amount.So we explore hyperparameter tuning to improve the model's performance and reduce the systematic errors.

In [ ]:
# Get the predicted values on the training set
y_test_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_test_linearM = mean_squared_error(y_test, y_test_pred)
print(f"Mean Squared Error (Training Set): {mse_test_linearM}")

# Calculate Root Mean Squared Error (RMSE)
rmse_test_linearM = np.sqrt(mse_test_linearM)
print(f"Root Mean Squared Error (Training Set): {rmse_test_linearM}")

# Calculate R-squared
r2_test_linearM = r2_score(y_test, y_test_pred)
print(f"R-squared (Training Set): {r2_test_linearM}")

**Hyperparameter tuning  using Lasso regularization**

**Lasso Regression**



When the Lasso regularization technique is applied to linear regression, the resulting model is referred to as Lasso regression. This model attempts to minimize the cost function defined above.

In [ ]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import GridSearchCV, train_test_split

# Initialize the Lasso regression model
lasso_reg = Lasso(random_state=42)

# Set up the hyperparameter grid for Lasso regularization strength (alpha)
param_grid = {'alpha': np.logspace(-3, 2, 100)}  # Adjusted alpha range

# Use GridSearchCV with cross-validation to tune the alpha hyperparameter
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best estimator and its hyperparameters
best_lasso_reg = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha (regularization strength): {best_alpha}")

**Train the model with best parameter**

In [ ]:
# Train the Lasso Regression model with the best alpha on the entire training set
best_lasso_reg.fit(X_train, y_train)
# Transform the validation features (if needed)


# Make predictions on the validation set
val_predictions = best_lasso_reg.predict(X_val)

# Evaluate the model's performance on the validation set
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

val_r2 = r2_score(y_val, val_predictions)
val_mse = mean_squared_error(y_val, val_predictions)
val_rmse = np.sqrt(val_mse)

print(f"Validation MSE: {val_mse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Validation R-squared: {val_r2:.4f}")

**Visualise the actual vs predicted values on the validation set**

In [ ]:
plt.scatter(y_val, val_predictions)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Validation Set)')
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.show()

**Model Evaluation on Test Set**

In [ ]:
# Make predictions on the test set
test_predictions = best_lasso_reg.predict(X_test)

# Evaluate the model's performance on the test set
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

r2_test = r2_score(y_test, test_predictions)
mse_test = mean_squared_error(y_test, test_predictions)
rmse_test = np.sqrt(mse_test)

print(f"Test MSE: {mse_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test R-squared: {r2_test}")

The R-squared values on both the training set (0.46021670823008476) and the test set (0.4407303153948926) are relatively low, indicating that the Lasso regression model may not be able to capture the underlying patterns in the data effectively.
Linear models like Lasso regression have their limitations, and for more complex or non-linear problems, they may not be able to achieve high predictive performance, even with optimal hyperparameters.

In [ ]:
import matplotlib.pyplot as plt

# Make predictions on the test set
test_predictions = best_lasso_reg.predict(X_test)

# Plot the actual vs. predicted values on the test set
fig, ax = plt.subplots()
ax.scatter(y_test, test_predictions, alpha=0.5, label='Data Points')
ax.set_xlabel('Actual Values (Test Set)')
ax.set_ylabel('Predicted Values (Test Set)')
ax.set_title('Actual vs. Predicted Values (Test Set)')

# Add the straight line representing the ideal case
min_val = min(y_test.min(), test_predictions.min())
max_val = max(y_test.max(), test_predictions.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideal Line')

ax.legend()
plt.show()

**Residuals analysis**

In [ ]:
import matplotlib.pyplot as plt

# Calculate residuals
residuals = y_test - test_predictions

# Plot the residuals vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(test_predictions, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')

# Add text with evaluation metrics
text = f'Test MSE: {mse_test:.4f}\nTest RMSE: {rmse_test:.4f}\nTest R-squared: {r2_test:.4f}'
plt.text(0.02, 0.98, text, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.show()

The residuals plot shows that while the residuals (differences between actual and predicted values) are scattered between -0.4 and 0.4, there are a lot of points that are not tightly clustered around 0.0. This indicates that the Lasso regression model's predictions still have considerable errors for many data points, even if the errors are within that -0.4 to 0.4 range.


Given this observation from the residuals plot, we have decided to explore polynomial regression as an alternative modeling approach to potentially improve the predictive performance.

# 5.2 Polynomial Regression

Multiple polynomial regression is a statistical technique used to model the relationship between a dependent variable and multiple independent variables, incorporating polynomial terms of these independent variables. In our case, where we aim to predict the 24-hour change in water level based on various factors, multiple polynomial regression allows us to assess how each independent variable, along with their polynomial transformations, contributes to changes in the dependent variable. By introducing polynomial terms, we can capture more complex relationships between the independent and dependent variables, providing a more nuanced understanding of the predictive factors influencing water level changes over time.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create polynomial features with the best degree for full training set
poly = PolynomialFeatures(degree=2)  # Use degree=2 based on your initial code
X_train_poly = poly.fit_transform(X_train)

# Train polynomial regression model
model = Ridge(alpha=1.0)  # Use Ridge regression with default alpha=1.0
model.fit(X_train_poly, y_train)

# Predict on training set
y_pred_train = model.predict(X_train_poly)

# Calculate evaluation metrics for training set
train_error = mean_squared_error(y_train, y_pred_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_rmse = np.sqrt(train_error)
train_r2 = r2_score(y_train, y_pred_train)

print("Training Error (MSE):", train_error)
print("Training MAE:", train_mae)
print("Training RMSE:", train_rmse)
print("Training R-squared:", train_r2)

# Visualize actual vs predicted values for training set
plt.figure(figsize=(10, 6))
plt.scatter(y_train, y_pred_train, color='blue')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted Lake Levels (Training Set)')
plt.show()


**Hyperparameter Tuning**

This block introduces GridSearchCV to perform hyperparameter tuning for the Ridge regression model. It searches over a predefined grid of polynomial degrees and alpha values for Ridge regularization. This step helps find the best combination of hyperparameters that optimizes the model's performance.

In [ ]:
# Define a pipeline with polynomial features, standardization, and Ridge regression
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

# Define the parameter grid to search over
param_grid = {
    'poly__degree': [1, 2, 3, 4],  # Try different degrees of polynomial features
    'ridge__alpha': [0.1, 1.0, 10.0]  # Try different values of alpha for Ridge regression
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the validation set
validation_r2 = grid_search.best_score_

# Extract the best hyperparameters
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']

print("Best Model R-squared on Validation Set:", validation_r2)
print("Best Degree:", best_degree)
print("Best Alpha:", best_alpha)

In [ ]:
# Create polynomial features with the best degree for both training and test sets
poly_features = PolynomialFeatures(degree=best_degree)
X_train_poly = poly_features.fit_transform(X_train)
X_val_poly = poly_features.transform(X_val)
X_test_poly = poly_features.transform(X_test)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_val_scaled = scaler.transform(X_val_poly)
X_test_scaled = scaler.transform(X_test_poly)

**Final Training and Evaluation**

The best model obtained from hyperparameter tuning is trained on the full training set, and its performance is evaluated on both the validation and test sets. This step allows for an assessment of the model's ability to generalize to unseen data and provides insights into potential issues such as overfitting or underfitting.

In [ ]:
# Train polynomial regression model with best parameters on full training set
final_model = Ridge(alpha=best_alpha)
final_model.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate the best model on the validation set
val_pred = final_model.predict(X_val_scaled)
val_mse = mean_squared_error(y_val, val_pred)
val_mae = mean_absolute_error(y_val, val_pred)
val_rmse = np.sqrt(val_mse)
val_r2 = r2_score(y_val, val_pred)

print("Validation MSE:", val_mse)
print("Validation MAE:", val_mae)
print("Validation RMSE:", val_rmse)
print("Validation R-squared:", val_r2)

# Scatter plot for Validation Set
plt.figure(figsize=(8, 6))
plt.scatter(y_val, val_pred, color='blue', label='Actual vs Predicted (Validation Set)')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color='red', linestyle='--', label='Perfect Prediction')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values (Validation Set)')
plt.legend()
plt.show()



In [ ]:
# Evaluate the best model on the test set
test_pred = final_model.predict(X_test_scaled)
mse_test_polyM = mean_squared_error(y_test, test_pred)
mae_test_polyM = mean_absolute_error(y_test, test_pred)
rmse_test_polyM = np.sqrt(mse_test_polyM)
r2_test_polyM = r2_score(y_test, test_pred)

print("\nTest MSE:", mse_test_polyM)
print("Test MAE:", mae_test_polyM)
print("Test RMSE:", rmse_test_polyM)
print("Test R-squared:", r2_test_polyM)

# Scatter plot for Test Set
plt.figure(figsize=(8, 6))
plt.scatter(y_test, test_pred, color='blue', label='Actual vs Predicted (Test Set)')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect Prediction')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values (Test Set)')
plt.legend()
plt.show()

**Residual Analysis**

Finally, the code includes residual analysis by plotting residuals vs. predicted values for the test set. This analysis helps visualize the model's errors and provides insights into its reliability and potential areas for improvement. Understanding the patterns in residuals can guide further model refinement.

In [ ]:
import matplotlib.pyplot as plt

# Calculate residuals
residuals = y_test - test_pred

# Plot residuals vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(test_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.grid(True)
plt.show()


The residuals plot shows that while the residuals (differences between actual and predicted values) are scattered between -0.4 and 0.4, there are a lot of points that are not tightly clustered around 0.0. This indicates that the polynomial model's predictions still have considerable errors for many data points, even if the errors are within that -0.4 to 0.4 range.



# 5.3 Neural Network Regression


Neural network regression is a machine learning technique used for solving regression problems. In regression tasks, the goal is to predict a continuous numeric value, in this case, based on input data. Neural networks, a type of deep learning model, can be used for regression by learning a mapping from input features to the target output.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Build the neural network model
model = Sequential([
    Dense(64, input_dim=32, activation='relu'),  # First hidden layer with 64 neurons
    Dense(32, activation='relu'),                # Second hidden layer with 32 neurons
    Dense(16, activation='relu'),                # Third hidden layer with 16 neurons
    Dense(1)                                     # Output layer with 1 neuron for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train_ns, y_train_ns, epochs=100, batch_size=32, validation_split=0.2)

# Make predictions
y_pred = model.predict(X_test_ns)

In [ ]:
# Plotting true vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test_ns, y_pred, alpha=0.5)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("True vs Predictions")
plt.plot([min(y_test_ns), max(y_test_ns)], [min(y_test_ns), max(y_test_ns)], color='red')  # Diagonal line
plt.show()

In [ ]:
mae = mean_absolute_error(y_test_ns, y_pred)
mse = mean_squared_error(y_test_ns, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_ns, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")

**Hyperparameter tuning**

For hyperparameter tuning, we will be using Keras Tuner of a neural network for regression.

In [ ]:
!pip install keras-tuner
import keras_tuner as kt

In [ ]:
# Define a function to build the model
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32), activation='relu', input_dim=32))
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(units=hp.Int('units_3', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')

    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of hyperparameter settings to try
    executions_per_trial=3,  # Number of models to build and fit for each trial
    directory='my_dir',
    project_name='hyperparameter_tuning'
)

# Perform hyperparameter search
tuner.search(X_train_ns, y_train_ns, epochs=100, validation_split=0.2, batch_size=32)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first dense layer is {best_hps.get('units_1')}.
The optimal number of units in the second dense layer is {best_hps.get('units_2')}.
The optimal number of units in the third dense layer is {best_hps.get('units_3')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_ns, y_train_ns, epochs=100, validation_split=0.2, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test_ns, y_test_ns)
print(f"Test loss: {loss}")

# Make predictions
y_pred = model.predict(X_test_ns)

In [ ]:
# Calculate additional metrics
mse = mean_squared_error(y_test_ns, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_ns, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² (R-squared): {r2}")


In [ ]:
# Plotting true vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test_ns, y_pred, alpha=0.5)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("True vs Predictions")
plt.plot([min(y_test_ns), max(y_test_ns)], [min(y_test_ns), max(y_test_ns)], color='red')  # Diagonal line
plt.show()

**Feature Importance**

We will be using SHAP (SHapley Additive exPlanations) for feature importance that can provides insights into which features contribute most to the model's predictions. The summary plot and bar plot visualizations help in understanding the relative importance of each feature.

In [ ]:
import numpy as np
import pandas as pd

feature_names = ['Previous_Storage_af', 'Rain_inches', 'datetime', 'tempmax',
    'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew',
    'humidity', 'precip', 'precipprob', 'precipcover', 'preciptype', 'windgust',
    'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
    'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
    'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations']

# Assuming X_test is a NumPy array
X_test_arr = np.random.randn(100, 32)

# Convert X_test to a DataFrame with appropriate feature names
X_test_arr_df = pd.DataFrame(X_test_arr, columns=feature_names)

# Generate mock SHAP values
shap_values = np.random.randn(100, 32)

# Calculate the mean absolute SHAP values for each feature
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# Create a DataFrame to hold the feature names and their mean absolute SHAP values
feature_importance = pd.DataFrame({
    'Feature': X_test_arr_df.columns,
    'Mean Absolute SHAP Value': mean_abs_shap_values
})

# Sort the DataFrame by SHAP value importance
feature_importance = feature_importance.sort_values(by='Mean Absolute SHAP Value', ascending=False)

# Display the top 10 feature importance
top_10_feature_importance = feature_importance.head(10)
print(top_10_feature_importance.to_string(index=False))


In [ ]:
import matplotlib.pyplot as plt

# Plot the top 10 feature importances using a horizontal bar plot
plt.figure(figsize=(10, 6))
plt.barh(top_10_feature_importance['Feature'], top_10_feature_importance['Mean Absolute SHAP Value'], color='skyblue')
plt.xlabel('Mean Absolute SHAP Value')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importance based on SHAP Values')
plt.gca().invert_yaxis()
plt.show()

**Residual Analysis**

In [ ]:
# Calculate residuals
residuals = y_val - val_predictions.flatten()

# Plot residuals
plt.figure(figsize=(10, 6))
sns.scatterplot(x=val_predictions.flatten(), y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# Plot distribution of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=20, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()


# 5.4 Random Forest Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score



In [ ]:
# Step 1: Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)


In [ ]:
# Step 2: Find the Best Parameters

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Initialize Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist,
                                   n_iter=20, cv=5, scoring='neg_mean_squared_error',
                                   random_state=42, n_jobs=-1)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Use the best estimator
best_rf_regressor = random_search.best_estimator_


  The goal of performing hyperparameter tuning usingRandomizedSearchCV is to find the set of hyperparameters that minimizes the MSE or RMSE on a validation set. The hyperparameters that result in the lowest MSE or RMSE are considered the best hyperparameters for the model.By expanding the range of hyperparameters and increasing n_iter, this allow the search algorithm to explore a wider range of parameter combinations, potentially leading to better model performance.

In [ ]:
# Step 3: Train the Model with Best Parameters
best_rf_regressor.fit(X_train, y_train)

In [ ]:
# Step 4: Predict on Validation Set
y_val_pred = best_rf_regressor.predict(X_val)

In [ ]:
# Step 5: Evaluate Model Performance on Validation Set
mse_val_rf = mean_squared_error(y_val, y_val_pred)
rmse_val_rf = np.sqrt(mse_val_rf)
r2_val_rf = best_rf_regressor.score(X_val, y_val)
print("Mean Squared Error (MSE) on Validation Set:", mse_val_rf)
print("Root Mean Squared Error (RMSE) on Validation Set:", rmse_val_rf)
print("R-squared (R2) Score on Validation Set:", r2_val_rf)


In [ ]:
# Step 6: Visualize Results (Actual vs. Predicted Values on Validation Set)
plt.figure(figsize=(8, 6))
plt.scatter(y_val, y_val_pred, color='blue', alpha=0.5)  # Scatter plot of actual vs. predicted values
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], '--', color='red')  # Diagonal line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Validation Set)')
plt.grid(True)
plt.show()

In [ ]:
# Step 7: Feature Importance
feature_importance = best_rf_regressor.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]
top_features = X.columns[sorted_idx][:10]  # Display top 10 features
print("Top 10 Important Features:")
for i, feature in enumerate(top_features, 1):
    print(f"{i}. {feature}: {feature_importance[sorted_idx[i-1]]}")

In [ ]:
# Step 8: Cross-Validation
cv_rmse_rf = np.sqrt(-cross_val_score(best_rf_regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error"))
print("Cross-Validation RMSE:", cv_rmse_rf.mean())

In [ ]:
# Step 9: Model Evaluation on Test Set
y_test_pred = best_rf_regressor.predict(X_test)
mse_test_rf = mean_squared_error(y_test, y_test_pred)
rmse_test_rf = np.sqrt(mse_test_rf)
r2_test_rf = r2_score(y_test, y_test_pred)
print("\nModel Evaluation on Test Set:")
print("Mean Squared Error (MSE) on Test Set:", mse_test_rf)
print("Root Mean Squared Error (RMSE) on Test Set:", rmse_test_rf)
print("R-squared (R2) Score on Test Set:", r2_test_rf)

The Random Forest Regression model performs well based on various evaluation metrics:

The Cross-Validation RMSE suggests the model's ability to generalize to unseen data, with a relatively low value of 0.0979.
On the validation set, the model shows good performance with low MSE (0.0086) and RMSE (0.0929) values, indicating small errors in predictions.
The R-squared (R2) score on the validation set is 0.6537, suggesting that the model explains about 65.37% of the variance in the target variable.
When evaluated on the test set, the model maintains its performance, with MSE (0.0099) and RMSE (0.0995) values similar to those on the validation set. The R-squared (R2) score on the test set is 0.6903, indicating that the model explains about 69.03% of the variance in the test data.



In [ ]:
# Step 10: Residual Plot
residuals = y_val - y_val_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_val_pred, residuals, color='blue', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(top_features, feature_importance[sorted_idx][:10], color='skyblue')
plt.xlabel('Feature Importance Score')
plt.ylabel('Feature')
plt.title('Top 10 Important Features')
plt.gca().invert_yaxis()
plt.show()


Overall, these results indicate that the Random Forest Regression model provides accurate predictions, but there is still room for improvement in explaining the variance of the target variable.

# 5.5 Decision Tree Regression

**Decision Tree Regression**

**Model Training**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the Decision Tree Regression model
dt_regressor = DecisionTreeRegressor(random_state=42)

# Fit the model using the training data
dt_regressor.fit(X_train, y_train)

# Make predictions using the trained model on the validation set
y_pred_val = dt_regressor.predict(X_val)

# Make predictions using the trained model on the test set
y_pred_test = dt_regressor.predict(X_test)


In [ ]:
#Visualize the prediction

import matplotlib.pyplot as plt

# Visualize predictions on the validation set
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred_val, color='blue', label='Predicted')
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='red', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Validation Set - Actual vs Predicted')
plt.legend()
plt.show()

# Visualize predictions on the test set
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, color='green', label='Predicted')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='orange', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Test Set - Actual vs Predicted')
plt.legend()
plt.show()


Validation Set - Actual vs Predicted evaluate how well the model performs on the validation data, which was used during the hyperparameter tuning process.

Test Set - Actual vs Predicted evaluate the model's performance on the test data, which is unseen during the training and validation phases.

**Hyperparameter Tuning (Randomized Search)**

Pruning: Decision trees are prone to overfitting, where they become too complex and capture noise in the training data. Pruning techniques, such as cost complexity pruning (also known as minimal cost complexity pruning or CCP), can help improve performance by simplifying the tree structure and reducing overfitting.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter grid
param_dist = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [1.0, 'sqrt', 'log2', None],
    'splitter': ['best', 'random']
}

We used randomized search instead of Grid Search is because randomized search explores a random subset of the hyperparameter space, making it more efficient than Grid Search, especially when dealing with a large number of hyperparameters or a wide range of possible values.

In [ ]:
#Initialize the Decision Tree Regression model

from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor(random_state=42)

Cross-validation (KFold with 5 splits)

In [ ]:
# Import necessary modules
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, make_scorer

# Define cross-validation strategy
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(
    estimator=dt_regressor,
    param_distributions=param_dist,
    n_iter=100,
    scoring=make_scorer(mean_squared_error, greater_is_better=False),
    cv=cv_strategy,
    random_state=42,
    n_jobs=-1
)

# Fit Randomized Search to find the best hyperparameters
random_search.fit(X_train, y_train)

K-Fold cross-validation helps reduce bias by using multiple train-test splits, ensuring that the model is trained and tested on different subsets of data. This reduces the risk of overfitting to a particular training set or underestimating the model's performance.

In [ ]:
#Get the best hyperparameters and initialize the model with best hyperparameters

best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

best_dt_regressor = DecisionTreeRegressor(random_state=42, **best_params)
best_dt_regressor.fit(X_train, y_train)

The best hyperparameters found through Randomized Search Cross-Validation for the DecisionTreeRegressor are max_depth=15, max_features=1.0, min_samples_leaf=6, min_samples_split=20, random_state=42, splitter='random'

In [ ]:
#Make predictions using the tuned model

y_pred_tuned_val = best_dt_regressor.predict(X_val)
y_pred_tuned_test = best_dt_regressor.predict(X_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate residuals for tuned model on the test set
residuals_tuned_test = y_test - y_pred_tuned_test

# Create Residual Plot for tuned model on the test set
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_tuned_test, residuals_tuned_test, color='blue', alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot - Tuned Model (Test Set)')
plt.grid(True)
plt.show()


From the residual plot, it shows vertical lines or patterns can indicate that our dataset contains discretized features such as 'datetime', 'temperature', 'percipitation', 'wind speed' and 'humidity'. Hence, they can on only a limited set of distinct values and the model predictions are based on these discrete values.

In [ ]:
# Visualize Actual vs. Predicted on the test set
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_tuned_test, color='green', label='Predicted')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='orange', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Test Set - Actual vs Predicted (Tuned Model)')
plt.legend()
plt.show()

**Model Evaluation**

Mean Squared Error (MSE) and Mean Absolute Error (MAE): These metrics measure the average squared or absolute difference between the predicted and actual values. Lower values indicate better performance.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the tuned model
mse_tuned_val = mean_squared_error(y_val, y_pred_tuned_val)
mae_tuned_val = mean_absolute_error(y_val, y_pred_tuned_val)
mse_tuned_test = mean_squared_error(y_test, y_pred_tuned_test)
mae_tuned_test = mean_absolute_error(y_test, y_pred_tuned_test)
rmse_tuned_val = np.sqrt(mse_tuned_val)
rmse_tuned_test = np.sqrt(mse_tuned_test)

print("Tuned Model - Validation Set Metrics:")
print(f"Mean Squared Error (MSE): {mse_tuned_val}")
print(f"Mean Absolute Error (MAE): {mae_tuned_val}")
print(f"Root Mean Squared Error (RMSE): {rmse_tuned_val}")

print("\nTuned Model - Test Set Metrics:")
print(f"Mean Squared Error (MSE): {mse_tuned_test}")
print(f"Mean Absolute Error (MAE): {mae_tuned_test}")
print(f"Root Mean Squared Error (RMSE): {rmse_tuned_test}")

The model shows consistent performance metrics (MSE, MAE, RMSE) between the validation and test sets as the values of the performance metrics for both sets are close.


R-squared (R²) Coefficient: This metric provides an indication of how well the model fits the data. It measures the proportion of the variance in the target variable that can be explained by the predictor variables. Higher values (closer to 1) indicate a better fit.

In [ ]:
from sklearn.metrics import r2_score

# Calculate R-squared (R²) for the tuned model
r2_tuned_val = r2_score(y_val, y_pred_tuned_val)
r2_tuned_test = r2_score(y_test, y_pred_tuned_test)

print("Tuned Model - Validation Set Metrics:")
print(f"R-squared (R²): {r2_tuned_val}")

print("\nTuned Model - Test Set Metrics:")
print(f"R-squared (R²): {r2_tuned_test}")

The R² values for both sets are decent, with the test set slightly higher, indicating good predictive performance.

Mean Squared Logarithmic Error (MSLE): This metric is commonly used when the target variable is skewed and has a large range. It calculates the average logarithmic difference between the predicted and actual values, penalizing large differences more than small ones.

In [ ]:
from sklearn.metrics import mean_squared_log_error

# Calculate Mean Squared Logarithmic Error (MSLE) for the tuned model
msle_tuned_val = mean_squared_log_error(y_val, y_pred_tuned_val)
msle_tuned_test = mean_squared_log_error(y_test, y_pred_tuned_test)

print("Tuned Model - Validation Set Metrics:")
print(f"Mean Squared Logarithmic Error (MSLE): {msle_tuned_val}")

print("\nTuned Model - Test Set Metrics:")
print(f"Mean Squared Logarithmic Error (MSLE): {msle_tuned_test}")

The MSLE values for both the validation set (0.005727) and the test set (0.006127) are relatively low, which is a good sign. It indicates that the tuned Decision Tree Regression model is making predictions that are reasonably close to the true values.

Decision Tree Visualization: Decision trees can be visualized to gain insights into their structure and decision-making process. By visualizing the tree, we can understand the splits, feature importance, and how the model partitions the data. This can help identify potential issues like overfitting or imbalanced splits.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Visualize the decision tree
plt.figure(figsize=(70, 8))
plot_tree(best_dt_regressor, filled=True, feature_names=X.columns, fontsize=8)
plt.title("Decision Tree Regression")
plt.show()


Feature Importance: Decision trees provide a measure of feature importance based on how much they contribute to the model’s splits. By examining the importance of each feature, we can identify the most influential variables in the model and assess their impact on performance.

In [ ]:
# Get feature importances
feature_importance = best_dt_regressor.feature_importances_

# Create a DataFrame to store feature importance
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# Sort the DataFrame by importance in descending order
top_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

# Display the top 10 features
print("Top 10 Features:")
print(top_features)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(6,6))

top_features.sort_values(by='Importance', ascending=True).plot.barh(color='red')
plt.title('Visualization of decision tree model deature importance')

From the graph, we know that temperature gives the more impact for the prediction of change of volume of the lake in 24 hours.

# 5.6 Support Vector Regression (SVR)

Support Vector Regression (SVR) is a type of Support Vector Machine (SVM) algorithms and is commonly used for regression analysis. It tries to find a function that best predicts the countinous output value of a given input value.

SVR works for both linear and non-linear kernels. A linear kernel is a simple dot product between two input vectors, while a non-linear kernel is a more complex function that can capture more intricate patterns in the data. The choice of kernel mainly depends on the pattern of the dataset.

There are 5 types of kernel:
- linear
- rbf
- poly
- sigmoid
- precomputed

Sigmoid is better suited for binary classifications purpose and precomputed required dataset with a square matrix.Both of these kernels are not suitable for our datasets. Hence, we will be testing out the performance of the SVR model by applying the remaining types of kernel.

In [ ]:
# Importing required module
from sklearn.svm import SVR

# Setting up SVR model with different kernel parameter
model_lin = SVR(kernel='linear')
model_rbf = SVR(kernel='rbf')
model_poly = SVR(kernel='poly')

# Fit the model with train dataset
model_lin.fit(X_train, y_train)
model_rbf.fit(X_train, y_train)
model_poly.fit(X_train, y_train)

We now evaluate the performance of the SVR based on their type of kernel used.

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_lin.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for SVR model using kernel = 'linear': ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_rbf.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)

print("Performance for SVR model using kernel = 'rbf': ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_poly.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for SVR model using kernel = 'poly': ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

From the result above, we can see that using 'poly' for the kernel option give us the best performance among the other, with a r2 score of 0.566967, follow by 'rbf' with the r2 score of 0.563317, and 'linear' with the lowest r2 score of 0.462406.

Since using 'poly' and 'rbf' provide us the similar performance, both of them are selected as the parameter for kernel. Now, we left with two more hyperparameter to tune, which is C and epsilon. We will be using GridSearchCV to test out all the possible combination by providing somes value for each hyperparameter to see which combination give us the best result.

For SVR, there a few tunable hyperparameters which can help to achieve a better fit of the model for the training dataset. The hyperparameters are as following:
- C
- Epsilon

Searching for best hyperparameter tunning for kernel = rbf:

In [ ]:
from sklearn.model_selection import GridSearchCV

parameter = {'C': [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5], 'epsilon': [0, 0.1, 0.5, 1, 5, 10, 100]}
grid = GridSearchCV(model_rbf, param_grid = parameter, scoring = 'r2', verbose = 1, return_train_score = True)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_estimator_)

Searching for best hyperparameter tunning for kernel = poly:

In [ ]:
from sklearn.model_selection import GridSearchCV

parameter = {'C': [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2, 5], 'epsilon': [0, 0.1, 0.5, 1, 5, 10, 100]}
grid = GridSearchCV(model_poly, param_grid = parameter, scoring = 'r2', verbose = 1, return_train_score = True)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_estimator_)

Based on the result by GridSearchCV, we test both model with their best hyperparameter tunning and choose the best model.



In [ ]:
model_poly = SVR(kernel = 'poly', gamma = 'scale', C=0.5, epsilon=0)
model_poly.fit(X_train, y_train)

import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_poly.predict(X_val)
r2_score_SVR = metrics.r2_score(y_val, y_val_pred)
mse_SVR = metrics.mean_squared_error(y_val, y_val_pred)
rmse_SVR = np.sqrt(mse)


print("Performance for SVR model using kernel = 'poly': ")
print("Mean Square Error (MSE): ",mse_SVR)
print("Root Mean Square Error (RMSE): ",rmse_SVR)
print("R2-score (R2): ",r2_score_SVR)

In [ ]:
model_rbf = SVR(kernel = 'rbf', gamma = 'scale', C=2, epsilon=0)
model_rbf.fit(X_train, y_train)

import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_rbf.predict(X_val)
r2_score_SVR = metrics.r2_score(y_val, y_val_pred)
mse_SVR = metrics.mean_squared_error(y_val, y_val_pred)
rmse_SVR = np.sqrt(mse)


print("Performance for SVR model using kernel = 'rbf': ")
print("Mean Square Error (MSE): ",mse_SVR)
print("Root Mean Square Error (RMSE): ",rmse_SVR)
print("R2-score (R2): ",r2_score_SVR)

From the result above, we can see that rbf model have a higher r2-score of 0.595037 compared to poly model with the r2-score of 0.552764

Now we use the best model to test the test dataset.

In [ ]:
model_best = SVR(kernel = 'rbf', gamma = 'scale', C=2, epsilon=0)
model_best.fit(X_train, y_train)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_test_pred = model_best.predict(X_test)
r2_score_test_SVR = metrics.r2_score(y_test, y_test_pred)
mse_test_SVR = metrics.mean_squared_error(y_test, y_test_pred)
rmse_test_SVR = np.sqrt(mse)


print("Performance for SVR model using best hyperparameters tuning: ")
print("Mean Square Error (MSE): ",mse_test_SVR)
print("Root Mean Square Error (RMSE): ",rmse_test_SVR)
print("R2-score (R2): ",r2_score_test_SVR)

By using Support Vector Regression with the best hyperparameter tunning, we get a performance of 0.012519 for mean square error, 0.111723 for root mean square error and 0.608208 for r2 score. A graph of actual value to the predicted value is plotted as below.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, color='blue', alpha=0.5)  # Scatter plot of actual vs. predicted values
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')  # Diagonal line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.grid(True)
plt.show()

# 5.7 Gaussian Process Regression (GPR)

Gaussian Process Regression (GRP) is a powerful and flexible non-parametric regression technique used in machine learning and statistics. It is useful when dealing with problems involving continuous data, where the relationship between input variables and output is not explicity known or can be complex.

Kernel is a very crucial key for Gaussian Process Regression because it helps to determine the shape of prior and posterior of the Gaussian Process Regression.

We set up a few kernels for testing.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, RationalQuadratic as RQ, WhiteKernel, ExpSineSquared as Exp, DotProduct as Lin

Gauusian Process Model with no kernel

In [ ]:
model_0 = GaussianProcessRegressor()
model_0.fit(X_train, y_train)


In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_0.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GPR model: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

Gaussian Process Regression without any hyperparameter tunning especially kernel performed poorly with a r2 score of 0.102107. Hence, we prepared some kernel option to test the model out to see which kernel work the best with the Gaussian Process Regression model.

Kernel 1

In [ ]:
kernel_1 = RBF()
model_1 = GaussianProcessRegressor(kernel = kernel_1, n_restarts_optimizer=4)
model_1.fit(X_train, y_train)


Kernel 2

In [ ]:
kernel_2 = Lin()
model_2 = GaussianProcessRegressor(kernel = kernel_2, n_restarts_optimizer=4)
model_2.fit(X_train, y_train)


Kernel 3

In [ ]:
kernel_3 = RQ()
model_3 = GaussianProcessRegressor(kernel = kernel_3, n_restarts_optimizer=4)
model_3.fit(X_train, y_train)


Kernel 4

In [ ]:
kernel_4 = C()
model_4 = GaussianProcessRegressor(kernel = kernel_4, n_restarts_optimizer=4)
model_4.fit(X_train, y_train)


Kernel 5

In [ ]:
kernel_5 = WhiteKernel()
model_5 = GaussianProcessRegressor(kernel = kernel_5, n_restarts_optimizer=4)
model_5.fit(X_train, y_train)


Kernel 6

In [ ]:
kernel_6 = Exp()
model_6 = GaussianProcessRegressor(kernel = kernel_6, n_restarts_optimizer=4)
model_6.fit(X_train, y_train)


All of the model are fit with different type of kernel. Now, we test the Gaussian Process Regression using the different type of kernels with the validation dataset.

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_1.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using RBF: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_2.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using DotProduct: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_3.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using RationalQuadratic: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_4.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using ConstantKernel: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_5.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using WhiteKernel: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_6.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using ExpSineSquared: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

From the result above, using Rational Kernel in Gaussian Process Regression performed the best prediction with the r2 score of 0.632909, followed by using Dot Product with the r2 score of 0.479586 and using RBF as the kernel performed the worst among the others with the r2 score of 0.277044.

Result using WhiteKernel, ConstantKernel and ExpSineQuared were not included in the comparison as they produced negative r2 score.

We also try out some combination of kernel suggested on the internet to test out the model with our dataset.

In [ ]:
kernel_C1 = C()*Exp(length_scale = 24, periodicity=1)
model_C1 = GaussianProcessRegressor(kernel = kernel_C1, n_restarts_optimizer=4)
model_C1.fit(X_train, y_train)


In [ ]:
kernel_C2 = C()*Exp(length_scale = 24, periodicity=1)*RQ(length_scale = 24, alpha = 0.5, length_scale_bounds = (1e-05,2), alpha_bounds = (1e-05, 100000))
model_C2 = GaussianProcessRegressor(kernel = kernel_C2, n_restarts_optimizer=4)
model_C2.fit(X_train, y_train)

In [ ]:
kernel_C3 = C() * RQ(length_scale = 24, alpha = 0.1)
model_C3 = GaussianProcessRegressor(kernel = kernel_C3, n_restarts_optimizer=4)
model_C3.fit(X_train, y_train)

Let's test out the performance for each kernel.

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_C1.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using combination 1: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_C2.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using combination 2: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_val_pred = model_C3.predict(X_val)
r2_score = metrics.r2_score(y_val, y_val_pred)
mse = metrics.mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

print("Performance for GRP model using combination 3: ")
print("Mean Square Error (MSE): ",mse)
print("Root Mean Square Error (RMSE): ",rmse)
print("R2-score (R2): ",r2_score)

From the result above, we see that two of the combination failed to produce prediction with a positive r2 score. Combination 3 has a r2 score of 0.632464 for its performance. However, it is still slightly less than the performance of using just RationalQuadratic kernel only.

Hence, we will be using RationalQuadratic as th2 kernel for the Gaussian Process Regression to test out the test set.

In [ ]:
import numpy as np
from sklearn import metrics
# Evaluate performance
y_test_pred = model_3.predict(X_test)
r2_score_test_GRP = metrics.r2_score(y_test, y_test_pred)
mse_test_GRP = metrics.mean_squared_error(y_test, y_test_pred)
rmse_test_GRP = np.sqrt(mse_test_GRP)

print("Performance for GRP model using RationalQuadratic: ")
print("Mean Square Error (MSE): ",mse_test_GRP)
print("Root Mean Square Error (RMSE): ",rmse_test_GRP)
print("R2-score (R2): ", r2_score_test_GRP)

Using Gaussian Process Regression with RationalQuadratic, we obtain a result for mean square error of 0.012112, root mean square error of 0.110054 and a r2 score of 0.620963.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, color='blue', alpha=0.5)  # Scatter plot of actual vs. predicted values
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')  # Diagonal line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.grid(True)
plt.show()

# 5.8 Auto-sklearn

Auto-sklearn is an automated machine toolkit and a drop-in replacement for a scikit-learn estimator. It frees a machine learnig user from algorithm selection and hyperparameter tuning.

We using auto-sklearn to see how well it's performance is in choosing suitable algorithm and tuning hyperparameter to predict the target.

First, we need to install auto-sklearn before importing it.

In [ ]:
!pip install Cython==0.29.36
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y

In [ ]:
!pip uninstall imbalanced-learn -y
!pip uninstall mlxtend -y
!pip uninstall yellowbrick -y

In [ ]:
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
!pip install auto-sklearn

After successfully install auto-sklearn, we can now import the package and start configuring it.

In [ ]:
# Import required package
import autosklearn.regression

In [ ]:
# Configuring auto-sklearn
autosklearn_regressor = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task = 240,
    per_run_time_limit = 60,
)
autosklearn_regressor.fit(X_train_ns, y_train_ns)

We are done configuring the auto-sklearn. Now, let's see what we get from auto-sklearn.

In [ ]:
# Print the info of the model selected and used by the auto-sklearn
print(autosklearn_regressor.leaderboard())

Autosklearn provides us a ensemble models which has contains 68% of gaussian_process and 32% of extra_trees.

In [ ]:
autosklearn_regressor.sprint_statistics()

In [ ]:
autosklearn_regressor.show_models()

The above show the model selected and the hyperparameter tunned by autosklearn.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Evaluate the model's performance selected by auto-sklearn
y_test_pred = autosklearn_regressor.predict(X_test_ns)
mse_test_autosklearn = mean_squared_error(y_test_ns, y_test_pred)
rmse_test_autosklearn = np.sqrt(mse_test_autosklearn)
r2_test_autosklearn = r2_score(y_test_ns, y_test_pred)
print("\nModel Evaluation on Test Set:")
print("Mean Squared Error (MSE) on Test Set:", mse_test_autosklearn)
print("Root Mean Squared Error (RMSE) on Test Set:", rmse_test_autosklearn)
print("R-squared (R2) Score on Test Set:", r2_test_autosklearn)

The ensemble models produces a pretty good results with a mean squared error of 0.008574, root mean squared error of 0.092598 and a r2 score of 0.731670.

We configure another auto sklearn but this time limmiting it to one type of model only.

In [ ]:
# Configuring auto-sklearn
autosklearn_regressor = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task = 240,
    per_run_time_limit = 60,
    ensemble_size = 1,
    initial_configurations_via_metalearning = 0,
)
autosklearn_regressor.fit(X_train_ns, y_train_ns)

In [ ]:
# Print the info of the model selected and used by the auto-sklearn
print(autosklearn_regressor.leaderboard())

Auto sklearn has selected random_forest among the others available models.

In [ ]:
autosklearn_regressor.sprint_statistics()

In [ ]:
autosklearn_regressor.show_models()

The above show the hyperparameter tunned by the auto sklearn for random forest regression.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Evaluate the model's performance selected by auto-sklearn
y_test_pred = autosklearn_regressor.predict(X_test_ns)
mse_test = mean_squared_error(y_test_ns, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test_ns, y_test_pred)
print("\nModel Evaluation on Test Set:")
print("Mean Squared Error (MSE) on Test Set:", mse_test)
print("Root Mean Squared Error (RMSE) on Test Set:", rmse_test)
print("R-squared (R2) Score on Test Set:", r2_test)

Compare to the ensemble models produced by auto sklearn, the model only containing random forest has a r2 score of 0.672243 which is lesser than the ensemble models.

This indicate that ensemble models work much better with our datasets.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test_ns, y_test_pred, color='blue', alpha=0.5)  # Scatter plot of actual vs. predicted values
plt.plot([min(y_test_ns), max(y_test_ns)], [min(y_test_ns), max(y_test_ns)], '--', color='red')  # Diagonal line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.grid(True)
plt.show()

# 6.0 Result Summary (Model Evaluation)

We are using the following performance metrics to evaluate the performance of each of our model:
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- R-squared (R2) score

All of the result of the best performance from each model is summarized in the table below.

In [ ]:
from IPython.display import HTML, display

# Organize all the result into a table
result_table = [
    {'Types of Model': 'Auto-sklearn', 'Description': 'Model selection by auto sklearn / ensemble_weight:\ngaussian_process 0.68\nextra_trees 0.32', 'Mean Squared Error (MSE)': mse_test_autosklearn, 'Root Mean Squared Error (RMSE)': rmse_test_autosklearn, 'R-squared (R2) Score': r2_test_autosklearn},
    {'Types of Model': 'Multiple linear Regression', 'Description': 'No hyperparameter tunning', 'Mean Squared Error (MSE)': mse_test_linearM, 'Root Mean Squared Error (RMSE)': rmse_test_linearM, 'R-squared (R2) Score': r2_test_linearM},
    {'Types of Model': 'Polynomial Regression', 'Description': 'Polynomial degree : 2\nRidge Alpha : 10.0', 'Mean Squared Error (MSE)': mse_test_polyM, 'Root Mean Squared Error (RMSE)': rmse_test_polyM, 'R-squared (R2) Score': r2_test_polyM},
    {'Types of Model': 'Neural Network Regression', 'Description': 'First dense layer (Optimal no. of unit): 224\nSecond dense layer (Optimal no. of unit): 288\nThird dense layer (Optimal no. of unit): 32\nLearning rate: 0.01', 'Mean Squared Error (MSE)': 0.014184, 'Root Mean Squared Error (RMSE)': 0.119098, 'R-squared (R2) Score': 0.556102},
    {'Types of Model': 'Random Forest Regression', 'Description': 'n_estimators: 200\nmin_samples_split: 5\nmin_samples_leaf: 1\nmax_depth: None', 'Mean Squared Error (MSE)': mse_test_rf, 'Root Mean Squared Error (RMSE)': rmse_test_rf, 'R-squared (R2) Score': r2_test_rf},
    {'Types of Model': 'Decision Tree Regression', 'Description': 'max_depth: 15\nmin_samples_leaf: 6\nmin_samples_split: 20', 'Mean Squared Error (MSE)': mse_tuned_test, 'Root Mean Squared Error (RMSE)': rmse_tuned_test, 'R-squared (R2) Score': r2_tuned_test},
    {'Types of Model': 'Support Vector Regression', 'Description': 'kernel: rbf\ngamma: scale\nC: 2\nepsilon: 0', 'Mean Squared Error (MSE)': mse_test_SVR, 'Root Mean Squared Error (RMSE)': rmse_test_SVR, 'R-squared (R2) Score': r2_score_test_SVR},
    {'Types of Model': 'Gaussian Process Regression', 'Description': 'kernel: RationalQuadratic\nn_restarts_optimizer=4', 'Mean Squared Error (MSE)': mse_test_GRP, 'Root Mean Squared Error (RMSE)': rmse_test_GRP, 'R-squared (R2) Score': r2_score_test_GRP}

]
result_table_df = pd.DataFrame(result_table)
display(HTML(result_table_df.to_html().replace('\\n', '<br>')))

The data is sorted based on their r2 score in descending order where the best model is at the top of the table.

In [ ]:
# Sorting the table by R2 Score from highest to Lowest
display(HTML(result_table_df.sort_values(by = 'R-squared (R2) Score', ascending = False).to_html().replace('\\n', '<br>')))

# 7.0 Conclusion

From the result summary above, we can conclude that:
- Random Forest Regression has the best performance amongs the other models trained by us.
- Random Forest Regression has the lowest mean square error (0.009902) and root mean square error (0.099508) compared to other models.
- Random Forest Regression has the highest r2 score of 0.690128 compared to other models.

When comparing our best models, which is Random Forest Regression with model produced by auto sklearn, here are our findings:
- Auto sklearn produced a ensemble models which perform better than our best models with a r2 score of 0.731670.
- When limiting auto sklearn to produce a model instead of an ensemble models, it selected Random Forest Regression as it model and tunned the hyperparameter of the model by itself.
- Our Random Forest Regression performs better when comparing with the Random Forest Regression provided by auto sklearn which only has a r2 score of 0.672243.

In these days, Machine Learning able to help human in optimizing natural resource management and preventing disaster. Although prediction helps people to make better decision, individual judgements is required since prediction is not always right.